# import

In [23]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
tf.__version__

'2.3.1'

# Checkpoint Function

In [24]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & Preprocessing Function

In [25]:
def load_mnist():
    (train_data,train_labels), (test_data,test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data,axis=-1) # [N,28,28] -> [N,28,28,1] : [batch_size,height,width,channel]
    test_data = np.expand_dims(test_data,axis=-1)   # [N,28,28] -> [N,28,28,1] axis=위치(-1=맨끝)
    
    train_data, test_data = normalize(train_data, test_data) # [0~255] -> [0~1]
    
    train_labels = to_categorical(train_labels,10) # [N,] -> [N,10]
    test_labels = to_categorical(test_labels,10) # [N,] -> [N,10] : One hot Encoding
    return train_data, train_labels, test_data, test_labels

def normalize(train_data,test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

# Performance Function

In [26]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels): # 정확도
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits,-1),tf.argmax(labels,-1))
    accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
    return accuracy

def grad(model, images, labels): # gradient
    with tf.GradientTape() as tape:
        loss = loss_fn(model,images,labels)
    return tape.gradient(loss,model.variables)

# Model Function

In [27]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init): # fully connected layer
    return tf.keras.layers.Dense(units=label_dim,use_bias=True,kernel_initializer=weight_init)
# unit = output으로 나갈 Channel이 몇개인가?
# use_bias = bias를 사용하는가 사용하지 않는가?
# kernel_initializer = weight 초기값. 

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model(class version)

In [28]:
class create_model_class(tf.keras.Model):
    def __init__(self,label_dim): # label_dim : Network의 Logit을 구할 때, 최종적으로 몇개의 Output을 낼 것인가?(여기선 10)
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() # Random Normal Initialization
        
        self.model = tf.keras.Sequential() # 층층이 쌓아나가는 것을 List처럼.
        self.model.add(flatten()) # [N,28,28,1] -> [N,784]
        # [N,784]로 바꿔주는 것은 나중에 fully connected layer를 이용할 것이기 때문. 
        # fully connected가 아니라
        
        for i in range(4):
            self.model.add(dense(512,weight_init))
            self.model.add(relu())
        
        self.model.add(dense(label_dim,weight_init))
        
    def call(self,x,training=None,mask=None):
        x = self.model(x)
        return(x)

# Create model(function version)

In [29]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.glorot_uniform()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(4):
        model.add(dense(512,weight_init))
        model.add(relu())
        
    model.add(dense(label_dim,weight_init))
    return model

# Define data & hyper-parameter

In [30]:
"""dataset"""
train_x, train_y, test_x, test_y = load_mnist()

"""parameters"""
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

"""Graph input using Dataset API"""
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = batch_size).\
    batch(batch_size,drop_remainder=True) # batch를 몇개만큼 네트워크에 던져 줄 것인가?
    # shuffle : 데이터셋을 섞는다. input 데이터보다 크게 설정하면 Random으로 shuffle한다.
    # prefetch : batch_size만큼 학습을 하고 있을 때, 미리 메모리에 batch_size만큼 올려놓으면 학습을 좀 더 빠르게 할 수 있다.
    # batch : batch를 몇개만큼 네트워크에 던져 줄 것인가?

test_dataset = tf.data.Dataset.from_tensor_slices((test_x,test_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = len(test_x)).\
    batch(len(test_x))

In [31]:
"""model"""
network = create_model_function(label_dim)

"""training"""
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

"""writer"""
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

# check point의 역할?
# network가 학습을 하다가 끊겼을 때, 다시 재학습을 하고 싶은 경우, 변경되었던 Weight를 불러내는 역할.
# 혹은 Test 이미지에 대해서 정확도를 보고 싶은 경우에도 활용할 수 있음.
# 학습되어 저장된 Weight를 불러오는것.
checkpoint_dir = os.path.join(checkpoint_dir,model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir,model_dir)
logs_dir = os.path.join(logs_dir,model_dir)

# Restore Check point & Start train or test phase

In [32]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exists
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print('[*] Load Success')
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print('[!] Load Failed...')
    
    # train phase
    with summary_writer.as_default(): # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx,(train_input,train_label) in enumerate(train_dataset):
                grads = grad(network,train_input,train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads,network.variables))
                
                train_loss = loss_fn(network,train_input,train_label)
                train_accuracy = accuracy_fn(network,train_input,train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network,test_input,test_label)
                
                tf.summary.scalar(name='train_loss',data=train_loss,step=counter)
                tf.summary.scalar(name='train_accuracy',data=train_accuracy,step=counter)
                tf.summary.scalar(name='test_accuracy',data=test_accuracy,step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f"\
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))

# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
[!] Load Failed...
Epoch: [ 0] [    0/  468] time: 0.7485, train_loss: 2.07613897, train_accuracy: 0.4922, test_accuracy: 0.2399
Epoch: [ 0] [    1/  468] time: 1.0727, train_loss: 1.99413466, train_accuracy: 0.5859, test_accuracy: 0.4685
Epoch: [ 0] [    2/  468] time: 1.4177, train_loss: 1.80680060, train_accuracy: 0.4531, test_accuracy: 0.3684
Epoch: [ 0] [    3/  468] time: 1.8057, train_loss: 1.68162763, train_accuracy: 0.5703, test_accuracy: 0.5392
Epoch: [ 0] [    4/  468] time: 2.0959, train_loss: 1.38778782, train_accuracy: 0.7031, test_accuracy: 0.6301
Epoch: [ 0] [    5/  468] time: 2.4231, train_loss: 1.03988969, train_accuracy: 0.7422, test_accuracy: 0.6535
Epoch: [ 0] [    6/  468] time: 2.7352, train_loss: 0.88045239, train_accuracy: 0.7891, test_accuracy: 0.7429
Epoch: [ 0] [    7/  468] time: 3.1591, train_loss: 1.03993082, train_accuracy: 0.6250, test_accuracy: 0.6382
Epoch: [ 0] [    8/  468] time: 3.4732, 

Epoch: [ 0] [   74/  468] time: 22.2670, train_loss: 0.22195032, train_accuracy: 0.9141, test_accuracy: 0.9297
Epoch: [ 0] [   75/  468] time: 22.5383, train_loss: 0.22798310, train_accuracy: 0.9375, test_accuracy: 0.9274
Epoch: [ 0] [   76/  468] time: 22.8184, train_loss: 0.19634068, train_accuracy: 0.9375, test_accuracy: 0.9315
Epoch: [ 0] [   77/  468] time: 23.0836, train_loss: 0.19363734, train_accuracy: 0.9375, test_accuracy: 0.9335
Epoch: [ 0] [   78/  468] time: 23.3529, train_loss: 0.10892683, train_accuracy: 0.9766, test_accuracy: 0.9367
Epoch: [ 0] [   79/  468] time: 23.6241, train_loss: 0.25103927, train_accuracy: 0.9531, test_accuracy: 0.9327
Epoch: [ 0] [   80/  468] time: 23.8974, train_loss: 0.25725305, train_accuracy: 0.9375, test_accuracy: 0.9243
Epoch: [ 0] [   81/  468] time: 24.1677, train_loss: 0.19301204, train_accuracy: 0.9609, test_accuracy: 0.9260
Epoch: [ 0] [   82/  468] time: 24.4349, train_loss: 0.18991430, train_accuracy: 0.9609, test_accuracy: 0.9286
E

Epoch: [ 0] [  148/  468] time: 43.5128, train_loss: 0.29953894, train_accuracy: 0.9375, test_accuracy: 0.9340
Epoch: [ 0] [  149/  468] time: 43.7970, train_loss: 0.17417264, train_accuracy: 0.9609, test_accuracy: 0.9344
Epoch: [ 0] [  150/  468] time: 44.1112, train_loss: 0.15346619, train_accuracy: 0.9453, test_accuracy: 0.9416
Epoch: [ 0] [  151/  468] time: 44.3944, train_loss: 0.11455342, train_accuracy: 0.9688, test_accuracy: 0.9477
Epoch: [ 0] [  152/  468] time: 44.6767, train_loss: 0.06372608, train_accuracy: 0.9922, test_accuracy: 0.9481
Epoch: [ 0] [  153/  468] time: 44.9663, train_loss: 0.15804657, train_accuracy: 0.9766, test_accuracy: 0.9449
Epoch: [ 0] [  154/  468] time: 45.2556, train_loss: 0.15590355, train_accuracy: 0.9453, test_accuracy: 0.9379
Epoch: [ 0] [  155/  468] time: 45.5617, train_loss: 0.20674834, train_accuracy: 0.9375, test_accuracy: 0.9314
Epoch: [ 0] [  156/  468] time: 45.8719, train_loss: 0.15826026, train_accuracy: 0.9453, test_accuracy: 0.9273
E

Epoch: [ 0] [  222/  468] time: 67.7858, train_loss: 0.07137810, train_accuracy: 0.9688, test_accuracy: 0.9549
Epoch: [ 0] [  223/  468] time: 68.1458, train_loss: 0.09363502, train_accuracy: 0.9766, test_accuracy: 0.9527
Epoch: [ 0] [  224/  468] time: 68.4779, train_loss: 0.15046966, train_accuracy: 0.9531, test_accuracy: 0.9526
Epoch: [ 0] [  225/  468] time: 68.8061, train_loss: 0.10944295, train_accuracy: 0.9609, test_accuracy: 0.9536
Epoch: [ 0] [  226/  468] time: 69.1582, train_loss: 0.14419942, train_accuracy: 0.9531, test_accuracy: 0.9523
Epoch: [ 0] [  227/  468] time: 69.6305, train_loss: 0.15664877, train_accuracy: 0.9531, test_accuracy: 0.9520
Epoch: [ 0] [  228/  468] time: 70.0813, train_loss: 0.09017463, train_accuracy: 0.9688, test_accuracy: 0.9515
Epoch: [ 0] [  229/  468] time: 70.4423, train_loss: 0.06826767, train_accuracy: 0.9766, test_accuracy: 0.9507
Epoch: [ 0] [  230/  468] time: 70.8133, train_loss: 0.08166562, train_accuracy: 0.9766, test_accuracy: 0.9492
E

Epoch: [ 0] [  296/  468] time: 99.7374, train_loss: 0.08618737, train_accuracy: 0.9766, test_accuracy: 0.9611
Epoch: [ 0] [  297/  468] time: 100.1693, train_loss: 0.16501588, train_accuracy: 0.9531, test_accuracy: 0.9604
Epoch: [ 0] [  298/  468] time: 100.5802, train_loss: 0.08330962, train_accuracy: 0.9844, test_accuracy: 0.9603
Epoch: [ 0] [  299/  468] time: 100.9548, train_loss: 0.13649842, train_accuracy: 0.9609, test_accuracy: 0.9593
Epoch: [ 0] [  300/  468] time: 101.3447, train_loss: 0.13198259, train_accuracy: 0.9609, test_accuracy: 0.9580
Epoch: [ 0] [  301/  468] time: 101.6888, train_loss: 0.06554657, train_accuracy: 0.9844, test_accuracy: 0.9564
Epoch: [ 0] [  302/  468] time: 102.0518, train_loss: 0.18824247, train_accuracy: 0.9688, test_accuracy: 0.9553
Epoch: [ 0] [  303/  468] time: 102.4264, train_loss: 0.09249121, train_accuracy: 0.9688, test_accuracy: 0.9513
Epoch: [ 0] [  304/  468] time: 102.7964, train_loss: 0.09236658, train_accuracy: 0.9766, test_accuracy: 

Epoch: [ 0] [  370/  468] time: 125.1707, train_loss: 0.08669791, train_accuracy: 0.9844, test_accuracy: 0.9515
Epoch: [ 0] [  371/  468] time: 125.4609, train_loss: 0.11396970, train_accuracy: 0.9766, test_accuracy: 0.9434
Epoch: [ 0] [  372/  468] time: 125.7741, train_loss: 0.12558033, train_accuracy: 0.9453, test_accuracy: 0.9366
Epoch: [ 0] [  373/  468] time: 126.0647, train_loss: 0.25320587, train_accuracy: 0.9219, test_accuracy: 0.9463
Epoch: [ 0] [  374/  468] time: 126.3878, train_loss: 0.06954384, train_accuracy: 0.9766, test_accuracy: 0.9607
Epoch: [ 0] [  375/  468] time: 126.7498, train_loss: 0.10611211, train_accuracy: 0.9844, test_accuracy: 0.9660
Epoch: [ 0] [  376/  468] time: 127.0889, train_loss: 0.09112594, train_accuracy: 0.9844, test_accuracy: 0.9626
Epoch: [ 0] [  377/  468] time: 127.4460, train_loss: 0.10897128, train_accuracy: 0.9531, test_accuracy: 0.9579
Epoch: [ 0] [  378/  468] time: 127.7591, train_loss: 0.06255494, train_accuracy: 0.9844, test_accuracy:

Epoch: [ 0] [  444/  468] time: 148.4968, train_loss: 0.07327801, train_accuracy: 0.9766, test_accuracy: 0.9602
Epoch: [ 0] [  445/  468] time: 148.7831, train_loss: 0.04858493, train_accuracy: 0.9844, test_accuracy: 0.9602
Epoch: [ 0] [  446/  468] time: 149.0703, train_loss: 0.13531955, train_accuracy: 0.9609, test_accuracy: 0.9610
Epoch: [ 0] [  447/  468] time: 149.3657, train_loss: 0.05247005, train_accuracy: 0.9766, test_accuracy: 0.9611
Epoch: [ 0] [  448/  468] time: 149.6639, train_loss: 0.03965034, train_accuracy: 0.9844, test_accuracy: 0.9620
Epoch: [ 0] [  449/  468] time: 149.9501, train_loss: 0.20681649, train_accuracy: 0.9453, test_accuracy: 0.9625
Epoch: [ 0] [  450/  468] time: 150.2419, train_loss: 0.05659284, train_accuracy: 0.9844, test_accuracy: 0.9609
Epoch: [ 0] [  451/  468] time: 150.5311, train_loss: 0.10270269, train_accuracy: 0.9609, test_accuracy: 0.9608
Epoch: [ 0] [  452/  468] time: 150.8153, train_loss: 0.06923039, train_accuracy: 0.9766, test_accuracy:

In [33]:
# test acc = 96.30